# 🎮 Batu Gunting Kertas - Original Teachable Machine Model

## 🎯 **Permainan Rock-Paper-Scissors 2 Pemain dengan MODEL ASLI**

### 🤖 **Menggunakan Original Teachable Machine Model:**
- ✅ **Keras Model** (`keras_model.h5`)
- ✅ **Original Labels** (`labels.txt`)
- ✅ **True AI Detection** (bukan heuristic)
- ✅ **Better Accuracy** dari training data

### 📋 **Cara Bermain:**
1. **Upload model files** (keras_model.h5, labels.txt)
2. **Run semua cell** (Ctrl+F9)
3. **Upload foto** untuk setiap pemain
4. **AI akan mendeteksi** dengan model asli!

---

## 🔧 **Setup & Upload Model Files**

### 📁 **Required Files:**
- `keras_model.h5` - Original Teachable Machine model
- `labels.txt` - Label file with Indonesian names

### 📤 **Upload kedua file terlebih dahulu:**

In [ ]:
from google.colab import files
import os

print("📤 Upload model files terlebih dahulu:")
print("\n1. Pilih file keras_model.h5")
print("2. Pilih file labels.txt")
print("\nUpload kedua file sekarang...")

# Upload model files
uploaded = files.upload()

if 'keras_model.h5' in uploaded and 'labels.txt' in uploaded:
    print("\n✅ Model files uploaded successfully!")
    print(f"   📁 keras_model.h5: {len(uploaded['keras_model.h5'])} bytes")
    print(f"   📁 labels.txt: {len(uploaded['labels.txt'])} bytes")
else:
    print("\n❌ Pastikan kedua file diupload (keras_model.h5 dan labels.txt)")

## 🔧 **Install Dependencies**

In [ ]:
# Install dependencies
print("🔧 Installing dependencies...")

!pip install tensorflow opencv-python numpy pillow matplotlib -q

import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from google.colab import files
import io
import warnings
warnings.filterwarnings('ignore')

print("✅ Dependencies installed successfully!")

## 🤖 **Teachable Machine Model Loader**

Compatible loader untuk original Teachable Machine model dengan TensorFlow compatibility fixes.

In [ ]:
# Custom compatible DepthwiseConv2D layer
class CompatibleDepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self, *args, **kwargs):
        # Remove 'groups' parameter if present (not supported in current TF)
        if 'groups' in kwargs:
            kwargs.pop('groups')
        super().__init__(*args, **kwargs)

class TeachableMachineModel:
    def __init__(self):
        self.model = None
        self.labels = []
        self.loaded = False

    def load_model_from_files(self, model_path='keras_model.h5', labels_path='labels.txt'):
        """Load the original Teachable Machine model with compatibility fixes"""
        try:
            print("🤖 Loading original Teachable Machine model...")

            # Load labels first
            if os.path.exists(labels_path):
                with open(labels_path, 'r') as file:
                    lines = [line.strip() for line in file.readlines()]
                    self.labels = []
                    for line in lines:
                        # Extract gesture name (remove number prefix like "0 batu")
                        gesture = line.split(' ', 1)[1] if ' ' in line else line
                        self.labels.append(gesture)
                print(f"✅ Labels loaded: {self.labels}")
            else:
                print(f"⚠️ Labels file not found: {labels_path}")
                self.labels = ['batu', 'gunting', 'kertas']  # Default fallback

            # Try multiple approaches to load the model
            model_loaded = False

            # Approach 1: With custom objects
            try:
                self.model = tf.keras.models.load_model(
                    model_path,
                    compile=False,
                    custom_objects={
                        'DepthwiseConv2D': CompatibleDepthwiseConv2D
                    }
                )
                print("✅ Model loaded with custom DepthwiseConv2D")
                model_loaded = True
            except Exception as e1:
                print(f"❌ Approach 1 failed: {str(e1)[:100]}...")

                # Approach 2: Register custom objects globally
                try:
                    tf.keras.utils.get_custom_objects()['DepthwiseConv2D'] = CompatibleDepthwiseConv2D
                    self.model = tf.keras.models.load_model(model_path, compile=False)
                    print("✅ Model loaded with global custom objects")
                    model_loaded = True
                except Exception as e2:
                    print(f"❌ Approach 2 failed: {str(e2)[:100]}...")

                    # Approach 3: Try loading without compilation
                    try:
                        self.model = tf.keras.models.load_model(model_path, compile=False)
                        print("✅ Model loaded without compilation")
                        model_loaded = True
                    except Exception as e3:
                        print(f"❌ Approach 3 failed: {str(e3)[:100]}...")

            if model_loaded:
                self.loaded = True
                print(f"🎉 Model successfully loaded! Input shape: {self.model.input_shape}")
                return True
            else:
                print("❌ All loading approaches failed")
                return False

        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False

    def preprocess_image(self, image):
        """Preprocess image for Teachable Machine model (224x224, normalized)"""
        try:
            # Convert PIL Image to numpy array
            if isinstance(image, Image.Image):
                img_array = np.array(image)
            else:
                img_array = image

            # Ensure RGB format
            if len(img_array.shape) == 3 and img_array.shape[2] == 4:
                img_array = img_array[:, :, :3]  # Remove alpha if present

            # Resize to 224x224 (Teachable Machine standard)
            img_resized = cv2.resize(img_array, (224, 224))

            # Convert RGB to BGR if needed (Teachable Machine models expect BGR)
            if len(img_resized.shape) == 3 and img_resized.shape[2] == 3:
                img_bgr = cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR)
            else:
                img_bgr = img_resized

            # Normalize pixel values to [0, 1]
            img_normalized = img_bgr / 255.0

            # Add batch dimension
            img_batch = np.expand_dims(img_normalized, axis=0)

            return img_batch

        except Exception as e:
            print(f"❌ Error preprocessing image: {e}")
            return None

    def predict(self, image):
        """Predict gesture using the loaded Teachable Machine model"""
        if not self.loaded:
            print("❌ Model not loaded!")
            return None, 0.0

        try:
            # Preprocess the image
            processed_image = self.preprocess_image(image)
            if processed_image is None:
                return None, 0.0

            # Make prediction
            predictions = self.model.predict(processed_image, verbose=0)

            # Get the predicted class and confidence
            predicted_class_index = np.argmax(predictions[0])
            confidence = float(predictions[0][predicted_class_index])

            # Get the label
            if predicted_class_index < len(self.labels):
                predicted_label = self.labels[predicted_class_index]
            else:
                predicted_label = 'unknown'

            return predicted_label, confidence

        except Exception as e:
            print(f"❌ Error making prediction: {e}")
            return None, 0.0

    def get_model_info(self):
        """Get information about the loaded model"""
        if self.loaded:
            info = {
                'loaded': True,
                'input_shape': self.model.input_shape,
                'output_shape': self.model.output_shape,
                'labels': self.labels,
                'num_classes': len(self.labels)
            }
        else:
            info = {
                'loaded': False,
                'error': 'Model not loaded'
            }
        return info

print("🤖 Teachable Machine Model Loader loaded!")

## 🔧 **Load Original Model**

In [ ]:
# Initialize and load the model
print("🚀 Loading your original Teachable Machine model...")
print("="*60)

tm_model = TeachableMachineModel()
success = tm_model.load_model_from_files('keras_model.h5', 'labels.txt')

print("\n" + "="*60)
if success:
    info = tm_model.get_model_info()
    print("🎉 MODEL LOADED SUCCESSFULLY!")
    print("="*60)
    print(f"📊 Model Info:")
    print(f"   Input Shape: {info['input_shape']}")
    print(f"   Output Shape: {info['output_shape']}")
    print(f"   Number of Classes: {info['num_classes']}")
    print(f"   Labels: {info['labels']}")
    print(f"\n✅ Original Teachable Machine model is ready!")
else:
    print("❌ MODEL LOADING FAILED!")
    print("="*60)
    print("💡 Make sure:")
    print("   1. keras_model.h5 is uploaded")
    print("   2. labels.txt is uploaded")
    print("   3. Files are in the correct format")

## 🎮 **Game Logic with Original Model**

In [ ]:
class BatuGuntingKertasGame:
    def __init__(self, model_instance):
        self.model = model_instance
        self.player1_choice = None
        self.player2_choice = None
        self.player1_image = None
        self.player2_image = None
        self.player1_confidence = 0
        self.player2_confidence = 0
        self.player1_score = 0
        self.player2_score = 0

    def get_emoji(self, choice):
        """Get emoji untuk setiap pilihan"""
        emoji_map = {
            'batu': '✊',
            'gunting': '✌️',
            'kertas': '✋'
        }
        return emoji_map.get(choice, '❓')

    def predict_gesture(self, image):
        """Predict gesture using the original Teachable Machine model"""
        if self.model and self.model.loaded:
            return self.model.predict(image)
        else:
            print("❌ Model not loaded!")
            return None, 0.0

    def determine_winner(self, player1_choice, player2_choice):
        """Menentukan pemenang"""
        if player1_choice == player2_choice:
            return 'tie', "Seri! Keduanya memilih yang sama."

        # Batu beats Gunting
        if player1_choice == 'batu' and player2_choice == 'gunting':
            return 'player1', '🗿 Batu menghancurkan Gunting!'

        if player1_choice == 'gunting' and player2_choice == 'batu':
            return 'player2', '🗿 Batu menghancurkan Gunting!'

        # Gunting beats Kertas
        if player1_choice == 'gunting' and player2_choice == 'kertas':
            return 'player1', '✌️ Gunting memotong Kertas!'

        if player1_choice == 'kertas' and player2_choice == 'gunting':
            return 'player2', '✌️ Gunting memotong Kertas!'

        # Kertas beats Batu
        if player1_choice == 'kertas' and player2_choice == 'batu':
            return 'player1', '✋ Kertas membungkus Batu!'

        if player1_choice == 'batu' and player2_choice == 'kertas':
            return 'player2', '✋ Kertas membungkus Batu!'

        return 'tie', 'Hasil tidak dapat ditentukan.'

    def reset_round(self):
        """Reset untuk ronde baru"""
        self.player1_choice = None
        self.player2_choice = None
        self.player1_image = None
        self.player2_image = None
        self.player1_confidence = 0
        self.player2_confidence = 0

# Initialize game with loaded model
if 'tm_model' in locals() and tm_model.loaded:
    game = BatuGuntingKertasGame(tm_model)
    print("✅ Game initialized with original Teachable Machine model!")
    print(f"🤖 Using model with labels: {tm_model.labels}")
else:
    print("❌ Cannot initialize game - model not loaded!")

## 📸 **Upload & AI Detection**

In [ ]:
def upload_player_image(player_num):
    """Upload dan proses gambar dengan original model"""
    print(f"\n📸 Upload foto untuk Pemain {player_num}:")
    print("📝 Cara pengambilan foto:")
    print("   • 🗿 **Batu**: Kepal tangan (fist)")
    print("   • ✌️ **Gunting**: 2 jari (peace sign)")
    print("   • ✋ **Kertas**: Tangan terbuka (open hand)")
    print("")

    uploaded = files.upload()

    if uploaded:
        # Ambil file yang diupload
        filename = list(uploaded.keys())[0]

        # Load image
        image = Image.open(io.BytesIO(uploaded[filename]))

        # Convert RGB jika perlu
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # AI Prediction dengan ORIGINAL MODEL
        print(f"🤖 Original Teachable Machine Model sedang menganalisis gambar Pemain {player_num}...")
        prediction, confidence = game.predict_gesture(image)

        # Simpan hasil
        if player_num == 1:
            game.player1_image = image
            game.player1_choice = prediction
            game.player1_confidence = confidence
        else:
            game.player2_image = image
            game.player2_choice = prediction
            game.player2_confidence = confidence

        # Tampilkan hasil
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.title(f'📸 Foto Pemain {player_num}', fontsize=14, weight='bold')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        emoji = game.get_emoji(prediction)
        plt.text(0.5, 0.6, emoji, fontsize=100, ha='center')
        plt.text(0.5, 0.3, prediction.upper(), fontsize=24, ha='center', weight='bold')
        plt.text(0.5, 0.1, f'Original AI Confidence: {confidence:.1%}', fontsize=16, ha='center', color='green')
        plt.text(0.5, 0.0, '🤖 Teachable Machine Model', fontsize=12, ha='center', style='italic')
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.axis('off')
        plt.title('🧠 Original AI Detection Result', fontsize=14, weight='bold')

        plt.tight_layout()
        plt.show()

        print(f"✅ Pemain {player_num}: {emoji} {prediction.upper()} (Original AI Confidence: {confidence:.1%})")
        return True
    else:
        print(f"❌ Tidak ada file yang diupload untuk Pemain {player_num}")
        return False

def show_results():
    """Tampilkan hasil pertandingan"""
    if game.player1_choice and game.player2_choice:
        # Tentukan pemenang
        winner, result_text = game.determine_winner(game.player1_choice, game.player2_choice)

        # Update skor
        if winner == 'player1':
            game.player1_score += 1
            winner_display = "🎉 PEMAIN 1 MENANG! 🎉"
            color = 'green'
        elif winner == 'player2':
            game.player2_score += 1
            winner_display = "🎉 PEMAIN 2 MENANG! 🎉"
            color = 'blue'
        else:
            winner_display = "🤝 SERI! 🤝"
            color = 'orange'

        # Tampilkan hasil
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle(f'🏆 {winner_display}', fontsize=28, weight='bold', color=color)

        # Pemain 1
        if game.player1_image:
            axes[0, 0].imshow(game.player1_image)
            axes[0, 0].set_title('👤 Pemain 1', fontsize=16, weight='bold')
            axes[0, 0].axis('off')

            emoji1 = game.get_emoji(game.player1_choice)
            axes[1, 0].text(0.5, 0.6, emoji1, fontsize=120, ha='center')
            axes[1, 0].text(0.5, 0.3, f'{game.player1_choice.upper()}', fontsize=20, ha='center', weight='bold')
            axes[1, 0].text(0.5, 0.1, f'AI Confidence: {game.player1_confidence:.1%}', fontsize=14, ha='center')
            axes[1, 0].text(0.5, 0.0, '🤖 Original Model', fontsize=12, ha='center', style='italic')
            axes[1, 0].set_xlim(0, 1)
            axes[1, 0].set_ylim(0, 1)
            axes[1, 0].axis('off')

        # Pemain 2
        if game.player2_image:
            axes[0, 1].imshow(game.player2_image)
            axes[0, 1].set_title('👥 Pemain 2', fontsize=16, weight='bold')
            axes[0, 1].axis('off')

            emoji2 = game.get_emoji(game.player2_choice)
            axes[1, 1].text(0.5, 0.6, emoji2, fontsize=120, ha='center')
            axes[1, 1].text(0.5, 0.3, f'{game.player2_choice.upper()}', fontsize=20, ha='center', weight='bold')
            axes[1, 1].text(0.5, 0.1, f'AI Confidence: {game.player2_confidence:.1%}', fontsize=14, ha='center')
            axes[1, 1].text(0.5, 0.0, '🤖 Original Model', fontsize=12, ha='center', style='italic')
            axes[1, 1].set_xlim(0, 1)
            axes[1, 1].set_ylim(0, 1)
            axes[1, 1].axis('off')

        plt.tight_layout()
        plt.show()

        # Tampilkan hasil text
        print(f"\n{'='*80}")
        print(f"🏆 HASIL PERTANDINGAN - ORIGINAL TEACHABLE MACHINE MODEL")
        print(f"{'='*80}")
        print(f"👤 Pemain 1: {game.get_emoji(game.player1_choice)} {game.player1_choice.upper()} (AI Confidence: {game.player1_confidence:.1%})")
        print(f"👥 Pemain 2: {game.get_emoji(game.player2_choice)} {game.player2_choice.upper()} (AI Confidence: {game.player2_confidence:.1%})")
        print(f"\n🎯 Hasil: {result_text}")
        print(f"\n📈 SKOR SAAT INI:")
        print(f"   👤 Pemain 1: {game.player1_score}")
        print(f"   👥 Pemain 2: {game.player2_score}")
        print(f"\n🤖 AI Model: Original Teachable Machine (keras_model.h5)")
        print(f"{'='*80}")

    else:
        print("❌ Belum semua pemain mengupload foto!")
        print("💡 Pastikan kedua pemain sudah mengupload foto sebelum melihat hasil.")

print("✅ Upload interface loaded with original model!")

## 👤 **Pemain 1 - Upload Foto**

*(Jalankan cell ini untuk Pemain 1)*

In [ ]:
# Reset ronde
game.reset_round()

# Upload untuk Pemain 1 dengan ORIGINAL MODEL
print(f"\n📈 SKOR TOTAL: P1: {game.player1_score} | P2: {game.player2_score}")
print("\n" + "="*60)
print("🤖 Menggunakan Original Teachable Machine Model")
print("="*60 + "\n")

player1_success = upload_player_image(1)

if player1_success:
    print("\n" + "="*50)
    print("✅ Pemain 1 berhasil! Sekarang giliran Pemain 2...")
    print("="*50)

## 👥 **Pemain 2 - Upload Foto**

*(Jalankan cell ini untuk Pemain 2)*

In [ ]:
# Upload untuk Pemain 2 dengan ORIGINAL MODEL
player2_success = upload_player_image(2)

if player2_success:
    print("\n" + "="*50)
    print("✅ Pemain 2 berhasil! Menentukan pemenang...")
    print("🧠 Menggunakan Original Teachable Machine AI")
    print("="*50)

## 🏆 **Hasil Pertandingan - Original AI**

*(Jalankan cell ini untuk melihat hasil dengan original model)*

In [ ]:
# Tampilkan hasil dengan ORIGINAL MODEL
show_results()

if game.player1_choice and game.player2_choice:
    print(f"\n🔄 Mau main lagi dengan original AI?")
    print("📍 Kembali ke cell 'Pemain 1 - Upload Foto' untuk ronde baru")

---

## 📖 **Panduan Lengkap - Original Model Version**

### 🎯 **Keunggulan Original Model:**
- ✅ **True AI**: Model yang Anda latih di Teachable Machine
- ✅ **Better Accuracy**: Berdasarkan training data Anda
- ✅ **Real Neural Network**: Bukan heuristic sederhana
- ✅ **Confidence Scoring**: Mencerminkan ketepatan model

### 🚀 **Cara Bermain dengan Original Model:**
1. **Upload Model Files**: keras_model.h5 & labels.txt
2. **Run All Cells**: Ctrl+F9 atau Runtime → Run all
3. **Upload Photos**: Pemain 1 & Pemain 2
4. **See Results**: AI detection dengan model asli!

### 🤖 **Model Information:**
- **Type**: Keras/TensorFlow model
- **Input**: 224x224 RGB images
- **Output**: 3 classes (batu, gunting, kertas)
- **Architecture**: Teachable Machine MobileNet

### 🎯 **Tips untuk Best Results:**
- **📸 Lighting**: Terang dan jelas
- **🖼️ Background**: Sederhana, tidak ramai
- **✋ Clear Gesture**: Tangan terlihat jelas
- **📏 Distance**: Tidak terlalu jauh/dekat
- **🎯 Similar to Training**: Photo style seperti saat training model

### 🏆 **Why This is Better:**
Original Teachable Machine model memberikan:
- **🧠 True Machine Learning**: Neural network yang trained
- **📊 Better Generalization**: Bekerja di berbagai kondisi
- **🎯 Pattern Recognition**: Mengenali fitur kompleks
- **📈 Real Confidence**: Based on actual model prediction

### 🔄 **Multiple Rounds:**
Game bisa dimainkan berkali-kali:
- Score terakumulasi
- Model tetap konsisten
- Results bisa dibandingkan

---

### 🎮 **Selamat Bermain dengan Original Teachable Machine Model!**

**🤖 AI Features:**
- ✅ Original Keras model (keras_model.h5)
- ✅ Indonesian labels (labels.txt)
- ✅ TensorFlow compatibility fixes
- ✅ Real neural network predictions

**Created with ❤️ using your original Teachable Machine model!**